In [60]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

##### Como baixar o chromedriver
Acesse o site: https://chromedriver.chromium.org/downloads
##### Case esteja usando a última versão
Acesse o site: https://googlechromelabs.github.io/chrome-for-testing/#stable

In [62]:
# https://googlechromelabs.github.io/chrome-for-testing/#stable
PATH = os.getcwd() + r'\Chomedriver\chromedriver.exe'

opcoes = Options()
# chrome_options.add_argument("--headless")
servico = webdriver.chrome.service.Service(PATH)

navegador = webdriver.Chrome(options=opcoes, service=servico)
navegador.get("http://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx")


In [ ]:
localizacao = (By.XPATH, '//*[@id="cphBody_btnFurtoCelular"]')
botao = WebDriverWait(navegador, 10).until(EC.presence_of_element_located(localizacao))
botao.click()

localizacao = (By.XPATH, '//*[@id="cphBody_lkAno10"]')
botao = WebDriverWait(navegador, 10).until(EC.presence_of_element_located(localizacao))
botao.click()

localizacao = (By.XPATH, '//*[@id="cphBody_lkMes2"]')
botao = WebDriverWait(navegador, 10).until(EC.presence_of_element_located(localizacao))
botao.click()

localizacao = (By.XPATH, '//*[@id="cphBody_ExportarBOLink"]')
botao = WebDriverWait(navegador, 10).until(EC.presence_of_element_located(localizacao))
botao.click()
